In [1]:
import os
print os.getcwd()

/Users/weidian1/Documents/GitHub/Udacity-DAND/P5-Intro to ML/Project5/final_project


In [2]:
#!/usr/bin/python

import sys
import pickle
sys.path.append("../tools/")
sys.path.append("../pylof-master/")

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

### Task 1: Select what features you'll use.
### features_list is a list of strings, each of which is a feature name.
### The first feature must be "poi".
features_list = ['poi','salary',''] # You will need to use more features

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)


/Users/weidian1/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Data Explore
Student response addresses the most important characteristics of the dataset and uses these characteristics to inform their analysis. Important characteristics include:

total number of data points  
allocation across classes (POI/non-POI)  
number of features used  
are there features with many missing values? etc.  

In [3]:
# 异常值， 去掉total
data_dict.pop('TOTAL', 0)

n_poi,n_non_poi = 0,0
for key in data_dict.keys():
    if data_dict[key]['poi']==1:
        n_poi+=1
    else:
        n_non_poi+=1
             
        
        
print "Total number of data points: ", len(data_dict)
print "Number of POI: %d, no. of non-POI: %d"% (n_poi, n_non_poi)
print "Number of features used: ",len(data_dict['METTS MARK']) # randomly pick one name, get the number of features.

Total number of data points:  145
Number of POI: 18, no. of non-POI: 127
Number of features used:  21


In [4]:
import pandas as pd
import numpy as np

df = pd.DataFrame.from_dict(data_dict,orient='index')


In [76]:
df.head(5)

,salary,to_messages,deferral_payments,total_payments,exercised_stock_options,bonus,restricted_stock,shared_receipt_with_poi,restricted_stock_deferred,total_stock_value,...,loan_advances,from_messages,other,from_this_person_to_poi,poi,director_fees,deferred_income,long_term_incentive,email_address,from_poi_to_this_person
ALLEN PHILLIP K,201955,2902,2869717,4484442,1729541,4175000,126027,1407,-126027,1729541,...,NaN,2195,152,65,False,NaN,-3081055,304805,phillip.allen@enron.com,47
BADUM JAMES P,NaN,NaN,178980,182466,257817,NaN,NaN,NaN,NaN,257817,...,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
BANNANTINE JAMES M,477,566,NaN,916197,4046157,NaN,1757552,465,-560222,5243487,...,NaN,29,864523,0,False,NaN,-5104,NaN,james.bannantine@enron.com,39
BAXTER JOHN C,267102,NaN,1295738,5634343,6680544,1200000,3942714,NaN,NaN,10623258,...,NaN,NaN,2660303,NaN,False,NaN,-1386055,1586055,NaN,NaN
BAY FRANKLIN R,239671,NaN,260455,827696,NaN,400000,145796,NaN,-82782,63014,...,NaN,NaN,69,NaN,False,NaN,-201641,NaN,frank.bay@enron.com,NaN


In [77]:
financial_features = ['salary', 'deferral_payments', 'total_payments', 'loan_advances', 'bonus'
                 , 'restricted_stock_deferred', 'deferred_income', 'total_stock_value', 'expenses'
                 , 'exercised_stock_options', 'other', 'long_term_incentive', 'restricted_stock'
                 , 'director_fees']
email_fatures = ['to_messages','from_poi_to_this_person', 'from_messages', 'from_this_person_to_poi'
                 , 'shared_receipt_with_poi']  # remove 'email_address'
def NaNs_to_0s(col):
    return [0 if ele=='NaN' else ele for ele in col]

def count_NAN(col):
    return sum(1 for ele in col if ele == 'NaN')

df[financial_features] = df[financial_features].apply(NaNs_to_0s, axis=0)
print df.apply(count_NAN, axis=0)    

salary                        0
to_messages                  59
deferral_payments             0
total_payments                0
exercised_stock_options       0
bonus                         0
restricted_stock              0
shared_receipt_with_poi      59
restricted_stock_deferred     0
total_stock_value             0
expenses                      0
loan_advances                 0
from_messages                59
other                         0
from_this_person_to_poi      59
poi                           0
director_fees                 0
deferred_income               0
long_term_incentive           0
email_address                34
from_poi_to_this_person      59
dtype: int64


> One thing that should be noted about the financial features is that the values were extracted from the "final_project/enron61702insiderpay.pdf" document. Cells in the table marked as "-" are parsed as NaN values in the saved data dictionary, but actually represent values of 0. https://discussions.udacity.com/t/how-to-start-the-final-project/177617/6  




In [78]:
# if the same person missing email_features
feas = email_fatures + ['poi','salary']
print df.ix[(df['to_messages']=='NaN')][feas]


                              to_messages from_poi_to_this_person  \
BADUM JAMES P                         NaN                     NaN   
BAXTER JOHN C                         NaN                     NaN   
BAY FRANKLIN R                        NaN                     NaN   
BAZELIDES PHILIP J                    NaN                     NaN   
BELFER ROBERT                         NaN                     NaN   
BERBERIAN DAVID                       NaN                     NaN   
BLAKE JR. NORMAN P                    NaN                     NaN   
BUTTS ROBERT H                        NaN                     NaN   
CHAN RONNIE                           NaN                     NaN   
CHRISTODOULOU DIOMEDES                NaN                     NaN   
CLINE KENNETH W                       NaN                     NaN   
CUMBERLAND MICHAEL S                  NaN                     NaN   
DETMERING TIMOTHY J                   NaN                     NaN   
DIMICHELE RICHARD G               

有且只有59个人email_features(except email_address) 全部missing，其余人没有任何feature missing。

FASTOW ANDREW S ;the chief financial officer of Enron Corporation
HIRKO JOSEPH :former co-chief executive officer of Enron Broadband Services (EBS)
KOPPER MICHAEL J:a former Enron Corporation finance executive, ran illicit partnerships allowing the company to conceal debt off its balance sheets.
WROBEL BRUCE : Senior Vice President of Strategic Development for Enron Broadband Services (EBS)

In [5]:
print data_dict['METTS MARK']
features_list = ['poi','salary', 'deferral_payments', 'total_payments', 'loan_advances', 'bonus'
                 , 'restricted_stock_deferred', 'deferred_income', 'total_stock_value', 'expenses'
                 , 'exercised_stock_options', 'other', 'long_term_incentive', 'restricted_stock', 'director_fees'
                 ,'to_messages', 'from_poi_to_this_person', 'from_messages', 'from_this_person_to_poi'
                 , 'shared_receipt_with_poi'] # You will need to use more features

{'salary': 365788, 'to_messages': 807, 'deferral_payments': 'NaN', 'total_payments': 1061827, 'exercised_stock_options': 'NaN', 'bonus': 600000, 'restricted_stock': 585062, 'shared_receipt_with_poi': 702, 'restricted_stock_deferred': 'NaN', 'total_stock_value': 585062, 'expenses': 94299, 'loan_advances': 'NaN', 'from_messages': 29, 'other': 1740, 'from_this_person_to_poi': 1, 'poi': False, 'director_fees': 'NaN', 'deferred_income': 'NaN', 'long_term_incentive': 'NaN', 'email_address': 'mark.metts@enron.com', 'from_poi_to_this_person': 38}


* financial features: ['salary', 'deferral_payments', 'total_payments', 'loan_advances', 'bonus', 'restricted_stock_deferred', 'deferred_income', 'total_stock_value', 'expenses', 'exercised_stock_options', 'other', 'long_term_incentive', 'restricted_stock', 'director_fees'] (all units are in US dollars)

* email features: ['to_messages', 'email_address', 'from_poi_to_this_person', 'from_messages', 'from_this_person_to_poi', 'shared_receipt_with_poi'] (units are generally number of emails messages; notable exception is ‘email_address’, which is a text string) 
* POI label: [‘poi’] (boolean, represented as integer)

In [22]:
### Store to my_dataset for easy export below.
my_dataset = data_dict

### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)


In [ ]:
# outliers lof
features_tuples = [tuple(row) for row in features]
from lof import outliers
lof = outliers(20, features_tuples)

for outlier in lof:
    print outlier["lof"],outlier["instance"]  

In [24]:
# split train test dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)
X_train = np.array(X_train)
y_train = np.array(y_train)
# TODO: Shuffle the data
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=0)

# tree selection
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
print "shape of X_train: ", X_train.shape

clf_tree = ExtraTreesClassifier(random_state=32)
clf_tree = clf_tree.fit(X_train, y_train)
print clf_tree.feature_importances_  

model_tree = SelectFromModel(clf_tree, prefit=True)
X_train_new = model_tree.transform(X_train)
print "shape of X_train_new: ", X_train_new.shape               


shape of X_train:  (100, 19)
[ 0.07597011  0.02106515  0.04194739  0.02004991  0.1376218   0.01235172
  0.06241739  0.04597266  0.1018082   0.08935221  0.05750866  0.04914737
  0.06703985  0.00171173  0.03369883  0.04670702  0.04292946  0.05600735
  0.03669318]
shape of X_train_new:  (100, 8)


In [43]:
#reselect features by feature_importances_
fea_imp = clf_tree.feature_importances_
features_new_index = sorted(range(len(fea_imp)), key=lambda k: fea_imp[k], reverse = True)[:8]
features_new_list=[]
for f in features_new_index:
    features_new_list.append(features_list[1:][f])
features_new_list = ['poi'] +features_new_list    

print features_new_list
print features_new_index

['poi', 'bonus', 'expenses', 'exercised_stock_options', 'salary', 'restricted_stock', 'deferred_income', 'other', 'from_this_person_to_poi']
[4, 8, 9, 0, 12, 6, 10, 17]


In [50]:
### Task 2: Remove outliers
### Task 3: Create new feature(s)
### Store to my_dataset for easy export below.
my_dataset = data_dict

### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)
labels = np.array(labels)
features = np.array(features)


In [56]:
type(features)

numpy.ndarray

In [ ]:
### Task 4: Try a varity of classifiers
### Please name your classifier clf for easy export below.
### Note that if you want to do PCA or other multi-stage operations,
### you'll need to use Pipelines. For more info:
### http://scikit-learn.org/stable/modules/pipeline.html

# Provided to give you a starting point. Try a variety of classifiers.
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()




In [52]:
### Task 5: Tune your classifier to achieve better than .3 precision and recall 
### using our testing script. Check the tester.py script in the final project
### folder for details on the evaluation method, especially the test_classifier
### function. Because of the small size of the dataset, the script uses
### stratified shuffle split cross validation. For more info: 
### http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedShuffleSplit.html

# Example starting point. Try investigating other evaluation techniques!
from sklearn.cross_validation import train_test_split
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)


In [60]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(features_train, labels_train)
clf.predict(features_test)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,
        0.,  0.,  0.,  0.,  0.])

In [61]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
print "recall_score: ",recall_score(labels_test, clf.predict(features_test))
print "precision_score: ",precision_score(labels_test, clf.predict(features_test))  


recall_score:  0.4
precision_score:  0.5


In [ ]:
### Task 6: Dump your classifier, dataset, and features_list so anyone can
### check your results. You do not need to change anything below, but make sure
### that the version of poi_id.py that you submit can be run on its own and
### generates the necessary .pkl files for validating your results.

dump_classifier_and_data(clf, my_dataset, features_list)